In [22]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns 
from surprise import Reader, Dataset, SVD, evaluate
sns.set_style("darkgrid")


ModuleNotFoundError: No module named 'surprise'

In [6]:
df1 = pd.read_csv("E:\major project\combined_data_1.txt", header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

In [7]:
df1['Rating'] = df1['Rating'].astype(float)

print('Dimension of the dataset: {}'.format(df1.shape))
print('Head')
print(df1.iloc[::5000000, :])

Dimension of the dataset: (24058263, 2)
Head
          Cust_Id  Rating
0              1:     NaN
5000000   2560324     4.0
10000000  2271935     2.0
15000000  1921803     2.0
20000000  1933327     3.0


In [8]:
df1

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0
...,...,...
24058258,2591364,2.0
24058259,1791000,2.0
24058260,512536,5.0
24058261,988963,3.0


In [9]:
df2 = pd.read_csv("E:\major project\combined_data_2.txt", header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])


In [10]:
df3 = pd.read_csv("E:\major project\combined_data_3.txt", header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])


In [11]:
df4 = pd.read_csv("E:\major project\combined_data_4.txt", header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

In [12]:
df2['Rating'] = df2['Rating'].astype(float)
df3['Rating'] = df3['Rating'].astype(float)
df4['Rating'] = df4['Rating'].astype(float)

In [14]:
df2

,Cust_Id,Rating
0,4500:,NaN
1,2532865,4.0
2,573364,3.0
3,1696725,3.0
4,1253431,3.0
...,...,...
26982297,2420260,1.0
26982298,761176,3.0
26982299,459277,3.0
26982300,2407365,4.0


In [15]:
df3

,Cust_Id,Rating
0,9211:,NaN
1,1277134,1.0
2,2435457,2.0
3,2338545,3.0
4,2218269,1.0
...,...,...
22605781,2339129,4.0
22605782,59005,4.0
22605783,1789683,5.0
22605784,1878798,1.0


In [16]:
df4

,Cust_Id,Rating
0,13368:,NaN
1,2385003,4.0
2,659432,3.0
3,751812,2.0
4,2625420,2.0
...,...,...
26851921,1790158,4.0
26851922,1608708,3.0
26851923,234275,1.0
26851924,255278,4.0


In [17]:
df = df1


In [18]:
df = df1.append(df2)


In [19]:
df

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0
...,...,...
26982297,2420260,1.0
26982298,761176,3.0
26982299,459277,3.0
26982300,2407365,4.0


In [20]:
df = df.append(df3)


In [21]:
df = df.append(df4)

In [22]:
df

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0
...,...,...
26851921,1790158,4.0
26851922,1608708,3.0
26851923,234275,1.0
26851924,255278,4.0


In [23]:
df.index = np.arange(0,len(df))

In [24]:
df

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0
...,...,...
100498272,1790158,4.0
100498273,1608708,3.0
100498274,234275,1.0
100498275,255278,4.0


In [25]:
RatingCount = df.groupby('Rating')['Rating'].agg(['count'])

In [27]:
RatingCount

,count
Rating,
1.0,4617990
2.0,10132080
3.0,28811247
4.0,33750958
5.0,23168232


In [28]:
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

In [29]:
movie_np = []
movie_id = 1

In [30]:
for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

In [31]:
# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

print('Movie numpy: {}'.format(movie_np))
print('Length: {}'.format(len(movie_np)))

Movie numpy: [1.000e+00 1.000e+00 1.000e+00 ... 1.777e+04 1.777e+04 1.777e+04]
Length: 100480507


In [44]:
movie_np

array([1.000e+00, 1.000e+00, 1.000e+00, ..., 1.777e+04, 1.777e+04,
       1.777e+04])

In [45]:
len(movie_np)

100483270

In [56]:
b = movie_np[:-2763]

In [58]:
len(b)

100480507

In [59]:
df = df[pd.notnull(df['Rating'])]
df['Movie_Id'] = b.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)

In [65]:
df

,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1
...,...,...,...
100498272,1790158,4.0,17770
100498273,1608708,3.0,17770
100498274,234275,1.0,17770
100498275,255278,4.0,17770


In [67]:
df.to_csv("E:\\major project\\rendred.txt")

MemoryError: Unable to allocate 767. MiB for an array with shape (100480507,) and data type int64

In [86]:
import random
sampled = random.sample(df, 21)

TypeError: Population must be a sequence or set.  For dicts, use list(d).

In [1]:
sample_data = df.sample(frac =.25)

NameError: name 'df' is not defined

In [5]:
df = pd.read_csv("E:\\major project\\rendred.txt")

In [6]:
f = ['count','mean']

In [8]:
df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)


In [13]:
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.6),0)


In [15]:
movie_benchmark

1006.0

In [17]:
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

In [18]:
drop_movie_list

Int64Index([    1,     2,     4,     7,     9,    10,    11,    12,    13,
               14,
            ...
            17753, 17754, 17755, 17757, 17759, 17760, 17765, 17766, 17767,
            17770],
           dtype='int64', name='Movie_Id', length=10656)

In [19]:
df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)



In [26]:
drop_cust_list

Int64Index([      8,      10,      25,      33,      42,      59,      83,
                 87,      94,     116,
            ...
            2649376, 2649379, 2649384, 2649388, 2649401, 2649404, 2649409,
            2649421, 2649426, 2649429],
           dtype='int64', name='Cust_Id', length=383899)

In [25]:
cust_benchmark = round(df_cust_summary['count'].quantile(0.8),0)
cust_benchmark
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index


In [42]:
df.shape

(100480507, 4)

In [48]:
df=df.drop(['Unnamed: 0'],axis=1)

In [49]:
df.dtypes

Cust_Id       int64
Rating      float64
Movie_Id      int64
dtype: object

In [50]:
df

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1
...,...,...,...
100480502,1790158,4.0,17770
100480503,1608708,3.0,17770
100480504,234275,1.0,17770
100480505,255278,4.0,17770


In [52]:
df.shape

(100480507, 3)

In [53]:
df = df[~df['Movie_Id'].isin(drop_movie_list)]
df = df[~df['Cust_Id'].isin(drop_cust_list)]

In [54]:
df.shape

(61640702, 3)

In [55]:
sample_data = df.sample(frac =.30)

In [5]:
sample_data

NameError: name 'sample_data' is not defined

In [59]:
df.to_csv("E:\\major project\\main_sample_data.txt")

In [3]:
df_sample = pd.read_csv("E:\major project\main_sample_data.txt")

In [4]:
df_sample

,Unnamed: 0,Cust_Id,Rating,Movie_Id
0,693,712664,5.0,3
1,694,1331154,4.0,3
2,697,656399,4.0,3
3,698,439011,1.0,3
4,700,1644750,3.0,3
...,...,...,...,...
61640697,100479577,1428223,4.0,17769
61640698,100479580,483107,4.0,17769
61640699,100479582,77664,2.0,17769
61640700,100479583,1922916,3.0,17769


In [6]:
df_sample=df_sample.drop(['Unnamed: 0'],axis=1)

In [7]:
df_sample

,Cust_Id,Rating,Movie_Id
0,712664,5.0,3
1,1331154,4.0,3
2,656399,4.0,3
3,439011,1.0,3
4,1644750,3.0,3
...,...,...,...
61640697,1428223,4.0,17769
61640698,483107,4.0,17769
61640699,77664,2.0,17769
61640700,1922916,3.0,17769


In [8]:
fraction_data = df_sample.sample(frac =.30)

In [9]:
fraction_data

,Cust_Id,Rating,Movie_Id
53928952,95762,5.0,15578
51064854,647933,2.0,14822
8405558,1555813,3.0,2594
31514561,1480900,3.0,9201
27614544,1607558,3.0,7970
...,...,...,...
8888573,304535,5.0,2782
13440930,387897,4.0,4109
59539359,408284,3.0,17169
20476690,1363315,5.0,6030


In [22]:
movieId_data_1=fraction_data.groupby('Movie_Id')['Movie_Id'].nunique()

In [18]:
movieId_data

Movie_Id
3          345
5          186
6          191
8         2129
16         709
         ...  
17762     7533
17763      477
17764    13027
17768      348
17769     1167
Name: Movie_Id, Length: 7114, dtype: int64

In [20]:
movieId_data.to_csv("E:\\major project\\movieId_data.txt")

In [21]:
fraction_data.to_csv("E:\\major project\\fraction_data.txt")

In [23]:
movieId_data_1

Movie_Id
3        1
5        1
6        1
8        1
16       1
        ..
17762    1
17763    1
17764    1
17768    1
17769    1
Name: Movie_Id, Length: 7114, dtype: int64

In [24]:
movieId_data_1.to_csv("E:\\major project\\movieId_data_1.txt")

In [3]:
df_movieid_genre = pd.read_csv("E:\\major project\\Final_Movie_Merged_15June2020.csv")

In [4]:
df_movieid_genre

,Movie_Id,Genre,Movie Name
0,732,Western,0.468055556
1,6807,Art&Foreign,8.5
2,2945,"Comedy, Romance",10
3,14249,"Comedy, Romance",10
4,6656,Action,10.5
...,...,...,...
7109,15233,Comedy,Zoolander
7110,13960,"Adventure, Drama",Zorba the Greek
7111,1210,"Action, Adventure, Comedy",Zorro
7112,17631,"Drama, History, War",Zulu


In [5]:
df_fraction_data = pd.read_excel("E:\\major project\\fraction_data.xlsx")

In [6]:
df_fraction_data

,Customer_ID,Rating,Movie_Id
0,95762,5,15578
1,647933,2,14822
2,1555813,3,2594
3,1480900,3,9201
4,1607558,3,7970
...,...,...,...
1048570,1132514,4,7617
1048571,1141334,5,3153
1048572,1352916,3,7523
1048573,1702603,3,10311


In [7]:
result= df_fraction_data.merge(df_movieid_genre,how='outer', left_on='Customer_ID', right_on='Movie_Id')

In [8]:
result

,Customer_ID,Rating,Movie_Id_x,Movie_Id_y,Genre,Movie Name
0,95762.0,5.0,15578.0,NaN,NaN,NaN
1,95762.0,4.0,17398.0,NaN,NaN,NaN
2,95762.0,4.0,4432.0,NaN,NaN,NaN
3,95762.0,4.0,15887.0,NaN,NaN,NaN
4,95762.0,3.0,12047.0,NaN,NaN,NaN
...,...,...,...,...,...,...
1055436,NaN,NaN,NaN,7058.0,Horror,Zombie
1055437,NaN,NaN,NaN,13960.0,"Adventure, Drama",Zorba the Greek
1055438,NaN,NaN,NaN,1210.0,"Action, Adventure, Comedy",Zorro
1055439,NaN,NaN,NaN,17631.0,"Drama, History, War",Zulu


In [9]:
result2 = pd.merge(left=df_movieid_genre, right=df_fraction_data, on=['Movie_Id'], how='inner')

In [11]:
result2

,Movie_Id,Genre,Movie Name,Customer_ID,Rating
0,732,Western,0.468055556,1759484,4
1,732,Western,0.468055556,2559642,1
2,732,Western,0.468055556,871436,3
3,732,Western,0.468055556,682407,4
4,732,Western,0.468055556,743813,3
...,...,...,...,...,...
1048570,10085,Drama,Zus & Zo,2426754,3
1048571,10085,Drama,Zus & Zo,2523426,4
1048572,10085,Drama,Zus & Zo,1793809,3
1048573,10085,Drama,Zus & Zo,2038142,2


In [13]:
fraction_data = result2.sample(frac =.5)

In [15]:
fraction_data

,Movie_Id,Genre,Movie Name,Customer_ID,Rating
877359,5939,"Action, Adventure, Fantasy",The Mummy Returns,2223831,2
9734,12140,"Action, Comedy, Crime",48 Hrs.,2283043,4
403987,7494,"Crime, Drama, Thriller",Jackie Brown: Collector's Series,1402612,3
307389,1144,Drama,Fried Green Tomatoes,2473795,5
437302,6445,"Adventure, Biography, Drama",Lawrence of Arabia,1705196,4
...,...,...,...,...,...
957671,17580,"Comedy, Crime, Thriller",The Whole Ten Yards,2583754,2
771396,11032,"Comedy, Drama",The Banger Sisters,938826,5
609624,6665,"Crime, Drama, Horror",Raising Cain,2264259,2
93665,14670,"Action, Adventure",Batman,900303,3


In [12]:
result2.to_csv("E:\\major project\\Final_Dataset")

In [17]:
import csv
reader=Reader()

NameError: name 'Reader' is not defined

In [33]:
pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
fraction_data.split(n_folds=3)

AttributeError: 'DataFrame' object has no attribute 'split'

In [69]:
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
sns.set_style("darkgrid")

In [49]:
reader=Reader()
data = Dataset.load_from_df(fraction_data[[ 'Customer_ID','Rating','Movie_Id']], reader)


In [52]:
data.split(n_folds=1)

AttributeError: 'DatasetAutoFolds' object has no attribute 'split'

In [70]:
svd=SVD()

In [57]:
cross_validate(svd, data, measures=['RMSE'])

{'test_rmse': array([10374.06294141, 10368.24798114, 10365.43001988, 10365.84820921,
        10390.56794479]),
 'fit_time': (31.698928833007812,
  32.89448380470276,
  28.737250804901123,
  31.17406415939331,
  28.238284826278687),
 'test_time': (1.4772775173187256,
  1.1492595672607422,
  1.1306936740875244,
  1.1616909503936768,
  0.9559545516967773)}

In [56]:
cross_validate

<function surprise.model_selection.validation.cross_validate(algo, data, measures=['rmse', 'mae'], cv=None, return_train_measures=False, n_jobs=1, pre_dispatch='2*n_jobs', verbose=False)>

In [62]:
df_785314 = fraction_data[(fraction_data['Customer_ID'] == 785314) & (fraction_data['Rating'] >= 3)]
df_785314 = df_785314.set_index('Movie_Id')
print(df_785314)

                      Genre                                    Movie Name  \
Movie_Id                                                                    
5516            Documentary                                American Movie   
1435       Animation,Comedy  Wallace & Gromit in Three Amazing Adventures   
11152                 Drama                            American History X   
11165     Adventure, Sci-Fi                         2001: A Space Odyssey   
14856        Action, Sci-Fi                              Dead Man Walking   
15246         Action,Sci-Fi                  Alien 3: Collector's Edition   

          Customer_ID  Rating  
Movie_Id                       
5516           785314       5  
1435           785314       4  
11152          785314       4  
11165          785314       4  
14856          785314       4  
15246          785314       3  


In [73]:
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate


In [77]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [83]:
df_785314 = df_785314.reset_index()
df_785314['Estimate_Score'] = df_785314['Movie_Id'].apply(lambda x: svd.predict(785314, x).est)

In [85]:
print(df_785314.head(10))

   Movie_Id              Genre                                    Movie Name  \
0      5516        Documentary                                American Movie   
1      1435   Animation,Comedy  Wallace & Gromit in Three Amazing Adventures   
2     11152              Drama                            American History X   
3     11165  Adventure, Sci-Fi                         2001: A Space Odyssey   
4     14856     Action, Sci-Fi                              Dead Man Walking   
5     15246      Action,Sci-Fi                  Alien 3: Collector's Edition   

   Customer_ID  Rating  Estimate_Score  
0       785314       5               5  
1       785314       4               5  
2       785314       4               5  
3       785314       4               5  
4       785314       4               5  
5       785314       3               5  


In [108]:
model = CFModel(78531,A,C)

NameError: name 'CFModel' is not defined

In [109]:
fraction_data.to_csv("E:\\major project\\Final_Fraction_Dataset")